# Conversion tool for the new pulse magnetometer to pmagpy format

All cells are (will be) included in the UF2MagIC notebook on https://github.com/dthallner/UF2MagIC after testing.

TO DO: <br>
check orientation parameters to make sure that it is converted correctly into pmagpy <br>
double check all units <br>

In [19]:
# import required packages
import os
import pandas as pd
import numpy as np

The notebook us currently only set up for AF or TH demag experiments!!! no combined TH-AF demags or intensity experiments yet!

To run, set a path to the directory with the files to convert (file_path) and define a site name for the samples (site_name). The notebook assumes all samples in the folder are from the same site -> this can be changed in the output files sites.txt (just add necessary sites) and samples.txt (change incorrect sites for each sample). It also assumes that sample == specimen. change in output files if that is not the case

Run all cells to get pmagpy output files<br>
1st cell defines path+site name<br>
2nd cell reads in all data<br>
3rd cell saves the output

In [20]:
# define path to folder with data. All .asc files in this folder will be combined to one measurements.txt output
file_path = r'G:\My Drive\__Postdoc\Misc\Side_quests\UF2MagIC\Testdata\Cryo' #path
site_name = 'Testsite' 

In [21]:
# Start new Magic contribution for locality

sites_dict = {'site': [], 'method_codes': [], 'citations': []}
samples_dict = {'site': [], 'sample': [],'method_codes': [], 'citations': [],
                'azimuth': [], 'dip': [], 'bed_dip_direction': [], 'bed_dip': [],'result_quality': []}
specimens_dict = {'specimen': [], 'sample': [],'method_codes': [],'experiments': [],'volume': [], 'citations': [],'result_quality':[]}
# redefine to get X Y and Z mag moment with standard deviations instead of dir_csd
measurement_dict = {'specimen': [],'method_codes': [],'dir_dec': [],'dir_inc': [],'magn_volume': [],'magn_moment': [], 'experiment': [],
                    'treat_temp':[], 'meas_temp': [],'treat_ac_field': [], 'treat_dc_field': [],'citations': [],
                    'treat_dc_field_phi': [],'instrument_codes': [],'sequence':[], 'measurement': [],
                    'treat_dc_field_theta': [],'treat_step_num':[],'quality': [],
                    'magn_x': [],'magn_x_sigma': [],'magn_y': [],'magn_y_sigma': [],'magn_z': [],'magn_z_sigma': []} 
                    #

for root, dirs, files in os.walk(file_path):
    for file in files:
        if file.endswith('.asc'):
            f = open(file_path+os.sep+file)
            data = f.readlines()
            f.close()
            with open(file_path+os.sep+'temp.txt','w') as fout: 
                for ii, line in enumerate(data):
                    if ii == 0: continue
                    if ii == 3:
                        line_new=[]
                        for ii in range(len(line)):
                            if not line[ii].strip(' ') == '':
                                line_new.append(line[ii].strip(' '))
                        fout.write(''.join(line_new).strip('\n')+'dir,time\n')
                    else:
                        fout.write(line)
                        
            f = open(file_path+os.sep+'temp.txt')
            data = f.readlines()
            f.close()
                
            header = data[1].split(',')
            data_df = pd.read_csv(file_path+os.sep+'temp.txt',skiprows=2,sep=(','),index_col=False)
            
            os.remove(file_path+os.sep+'temp.txt')
            for key in data_df.keys(): 
                data_df.rename(columns={key: key.strip('\t ')}, inplace=True)
            sName = str(header[0])
            demag_type = ''
            
            samples_dict['site'].append(str(site_name))
            samples_dict['sample'].append(str(sName))
            samples_dict['citations'].append('This study')      
            samples_dict['result_quality'].append('g') # all our measurements are 'g'ood!!
            samples_dict['azimuth'].append(header[4])
            if int(header[8]) == 0: # if not overturned
                plunge = header[5]
            else:
                plunge = str(float(header[5])*-1)
            samples_dict['dip'].append(str(float(plunge)-90))
            dip_azimuth = header[6]
            samples_dict['bed_dip_direction'].append(str(float(dip_azimuth)+0))
            dip_angle = header[7]
            samples_dict['bed_dip'].append(dip_angle)

                         
            specimens_dict['specimen'].append(sName) 
            specimens_dict['sample'].append(sName)
            specimens_dict['volume'].append(float(header[1])* 10**-6) # CC to m3
            specimens_dict['citations'].append('This study')
            specimens_dict['result_quality'].append('g')
            
            for ii in range(len(data_df)):
                measurement_dict['specimen'].append(sName)
                if data_df.loc[data_df.index[ii],'DEMAG'] == 'NRM':
                    if ii > 0 and data_df.loc[data_df.index[ii-1],'DEMAG'] == 'NRM':
                        measurement_dict['experiment'].append(sName+'_'+'LP-DIR')
                        measurement_dict['measurement'].append(sName+'_'+'LP-DIR'+'-'+str(ii))
                    measurement_dict['method_codes'].append('LT-NO')
                    measurement_dict['treat_temp'].append('273')
                    measurement_dict['treat_ac_field'].append('0') #T
                elif 'mT' in data_df.loc[data_df.index[ii],'DEMAG']:
                    demag_type = 'AFD'
                    measurement_dict['method_codes'].append('LT-AF-Z')
                    if ii > 0 and data_df.loc[data_df.index[ii-1],'DEMAG'] == 'NRM':
                        measurement_dict['experiment'].append(sName+'_'+'LP-DIR-AF')
                        measurement_dict['measurement'].append(sName+'_'+'LP-DIR-AF'+'-'+str(ii))
                    measurement_dict['experiment'].append(sName+'_'+'LP-DIR-AF')
                    measurement_dict['measurement'].append(sName+'_'+'LP-DIR-AF'+'-'+str(ii+1))
                    measurement_dict['treat_temp'].append('273')
                    measurement_dict['treat_ac_field'].append(str(float(data_df.loc[data_df.index[ii],'DEMAG'].strip('mT'))/10**3)) #T
                else:
                    demag_type = 'THD'
                    measurement_dict['method_codes'].append('LT-T-Z')
                    if ii > 0 and data_df.loc[data_df.index[ii-1],'DEMAG'] == 'NRM':
                        measurement_dict['experiment'].append(sName+'_'+'LP-DIR-T')
                        measurement_dict['measurement'].append(sName+'_'+'LP-DIR-T'+'-'+str(ii))
                    measurement_dict['experiment'].append(sName+'_'+'LP-DIR-T')
                    measurement_dict['measurement'].append(sName+'_'+'LP-DIR-T'+'-'+str(ii+1))
                    measurement_dict['treat_temp'].append(str(float(data_df.loc[data_df.index[ii],'DEMAG'])+273)) # K
                    measurement_dict['treat_ac_field'].append('0')
                    
                measurement_dict['dir_dec'].append(data_df.loc[data_df.index[ii],'CD'])
                measurement_dict['dir_inc'].append(data_df.loc[data_df.index[ii],'CI'])
                
                measurement_dict['magn_volume'].append(data_df.loc[data_df.index[ii],'J'] * 10**-3) # emu/cc to A/m
                measurement_dict['magn_moment'].append(data_df.loc[data_df.index[ii],'M'] * 10**3) # emu to Am^2
                measurement_dict['magn_x'].append(data_df.loc[data_df.index[ii],'X'] * 10**3)
                measurement_dict['magn_x_sigma'].append(data_df.loc[data_df.index[ii],'SX'] * 10**3)
                measurement_dict['magn_y'].append(data_df.loc[data_df.index[ii],'Y'] * 10**3)
                measurement_dict['magn_y_sigma'].append(data_df.loc[data_df.index[ii],'SY'] * 10**3)
                measurement_dict['magn_z'].append(data_df.loc[data_df.index[ii],'Z'] * 10**3)
                measurement_dict['magn_z_sigma'].append(data_df.loc[data_df.index[ii],'SZ'] * 10**3)
                
                
                measurement_dict['meas_temp'].append('273') # [K] can be changed to room temp id needed
                measurement_dict['treat_dc_field'].append('')
                measurement_dict['citations'].append('This study')
                measurement_dict['treat_dc_field_phi'].append('')
                measurement_dict['instrument_codes'].append('2G')
                measurement_dict['sequence'].append(str(ii+1))
                measurement_dict['treat_dc_field_theta'].append('')
                measurement_dict['treat_step_num'].append(str(ii+1))
                # assuming that in case of double-measurement the first measurement is 'b'ad and the repeat is 'g'ood
                if not ii == 0 and data_df.loc[data_df.index[ii-1],'DEMAG'] == data_df.loc[data_df.index[ii],'DEMAG']:
                    measurement_dict['quality'].append('b')
                else:
                    measurement_dict['quality'].append('g')
                
                
            if demag_type == 'AFD':
                samples_dict['method_codes'].append('LP-DIR-AF')
            elif demag_type == 'THD':
                samples_dict['method_codes'].append('LP-DIR-T')
            else:
                samples_dict['method_codes'].append('LP-DIR')
                
            if demag_type == 'AFD':
                specimens_dict['method_codes'].append('LP-DIR-AF')
                specimens_dict['experiments'].append(sName+'_LP-DIR-AF')
            elif demag_type == 'THD':
                specimens_dict['method_codes'].append('LP-DIR-T')
                specimens_dict['experiments'].append(sName+'_LP-DIR-T')
            else:
                specimens_dict['method_codes'].append('LP-DIR')
                specimens_dict['experiments'].append(sName+'_LP-DIR')

sites_dict['site'].append(str(site_name))
if demag_type == 'AFD':
    sites_dict['method_codes'].append('LP-DIR-AF')
elif demag_type == 'THD':
    sites_dict['method_codes'].append('LP-DIR-T')
else:
    sites_dict['method_codes'].append('LP-DIR')
sites_dict['citations'].append('This study')
print('Done!')

Done!


In [22]:
# save files for demag_gui

with open('./sites.txt', 'w') as sites_file:
    sites_file.write('tab\tsites\n')
sites_df = pd.DataFrame(sites_dict)
sites_df.to_csv('./sites.txt',mode='a',index=False,sep='\t')
    
with open('./samples.txt', 'w') as samples_file:
    samples_file.write('tab\tsamples\n')
samples_df = pd.DataFrame(samples_dict)
samples_df.to_csv('./samples.txt',mode='a',index=False,sep='\t')

with open('./specimens.txt', 'w') as specimens_file:
    specimens_file.write('tab\tspecimens\n')
specimens_df = pd.DataFrame(specimens_dict)
specimens_df.to_csv('./specimens.txt',mode='a',index=False,sep='\t')

with open('./measurements.txt', 'w') as measurements_file:
    measurements_file.write('tab\tmeasurements\n')
measurement_df = pd.DataFrame(measurement_dict)
measurements_df = measurement_df[(~measurement_df['specimen'].isna())]
#measurement_df = measurement_df.sort_values(by=['specimen','sequence'])
measurement_df.to_csv('./measurements.txt',mode='a',index=False,sep='\t')

print('Files for demag GUI saved to', os.getcwd())

Files for demag GUI saved to G:\My Drive\__Postdoc\Misc\Side_quests\UF2MagIC
